In [1]:
def read_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = f.read()
        assert data, f"Файл {file_path} пустой или не удалось cчитать данные."
        return data

text = read_data('C:/Tokenizer/DataSets/ru.txt')

In [2]:
import re

def clean_text(text):
    text = re.sub(r"[^а-яё\s]", "", text.lower())
    text = ' '.join(text.split())
    return text

corpus = clean_text(text)

In [3]:
if corpus is not None:
    print("Первые 100 символов из ру:")
    print(corpus[:100])

Первые 100 символов из ру:
я думал я буду читать те свои стихи которые связаны с темой юности и зрелости я даже немного удивилс


In [4]:
from collections import defaultdict

vocab = defaultdict(int)
for word in corpus.split():
    for char in word:
        vocab[char] += 1
    vocab['_ed'] += 1

In [5]:
print(len(vocab))
vocab

34


defaultdict(int,
            {'я': 342793,
             '_ed': 3064170,
             'д': 505746,
             'у': 397444,
             'м': 640711,
             'а': 1251411,
             'л': 675358,
             'б': 268953,
             'ч': 272410,
             'и': 1249335,
             'т': 1200315,
             'ь': 318961,
             'е': 1380256,
             'с': 861114,
             'в': 702425,
             'о': 1867683,
             'х': 161763,
             'к': 515989,
             'р': 728335,
             'ы': 371311,
             'з': 279298,
             'н': 1071319,
             'й': 180365,
             'ю': 131659,
             'ж': 163408,
             'г': 251697,
             'п': 435103,
             'щ': 63687,
             'ш': 108749,
             'ё': 48462,
             'ц': 66405,
             'э': 112219,
             'ф': 44082,
             'ъ': 4417})

In [6]:
words = defaultdict(int)

for word in corpus.split():
    symbols = list(word) + ['_ed']
    words[' '.join(symbols)] += 1

In [7]:
for i, (key, value) in enumerate(words.items()):
    if i < 5:
        print(f"'{key}': {value}")
    else:
        break

'я _ed': 51449
'д у м а л _ed': 415
'б у д у _ed': 645
'ч и т а т ь _ed': 254
'т е _ed': 1781


In [8]:
def get_stats(words):
    pairs = defaultdict(int)
    for word, freq in words.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            if symbols[i+1] != '_ed':
                pair = (symbols[i], symbols[i+1])
                pairs[pair] += freq
    return pairs

In [9]:
pairs = get_stats(words)
print(sorted(((v,k) for k,v in pairs.items()), reverse=True))

[(313608, ('т', 'о')), (231425, ('с', 'т')), (204957, ('н', 'о')), (204786, ('н', 'а')), (175042, ('н', 'и')), (165335, ('е', 'н')), (163421, ('п', 'о')), (162727, ('р', 'а')), (155561, ('к', 'о')), (150145, ('р', 'о')), (139773, ('о', 'т')), (139714, ('н', 'е')), (138165, ('т', 'ь')), (136865, ('о', 'в')), (136482, ('л', 'и')), (136475, ('п', 'р')), (125513, ('т', 'е')), (121966, ('т', 'а')), (121174, ('е', 'т')), (117139, ('о', 'с')), (116513, ('о', 'л')), (114543, ('р', 'е')), (114230, ('в', 'о')), (112119, ('в', 'а')), (110807, ('о', 'р')), (110454, ('к', 'а')), (109239, ('е', 'р')), (109214, ('г', 'о')), (106537, ('д', 'е')), (106124, ('е', 'с')), (105025, ('о', 'м')), (101762, ('и', 'т')), (100594, ('а', 'т')), (100590, ('о', 'д')), (100580, ('а', 'л')), (98860, ('а', 'н')), (97348, ('м', 'е')), (96354, ('о', 'г')), (94823, ('о', 'н')), (94654, ('л', 'ь')), (94277, ('е', 'л')), (93437, ('м', 'о')), (92672, ('л', 'е')), (91480, ('э', 'т')), (91074, ('е', 'м')), (91063, ('т', 'и'))

In [10]:
top_pair = max(pairs, key=pairs.get)
top_pair

('т', 'о')

In [11]:
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = ' '.join(pair)
    pattern = re.compile(r'(?<!\S)' + re.escape(bigram) + r'(?!\S)')
    for word_in in v_in:
        word_out = pattern.sub(''.join(pair), word_in)
        v_out[word_out] = v_in[word_in]
    return v_out

In [12]:
vocab_size = 1000
num_merges = vocab_size - len(vocab)
for i in range(num_merges):
    pairs = get_stats(words)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab[''.join(best)] = pairs[best]
    words = merge_vocab(best, words)
    if i % 10 == 0:
        print(f'Итерация {i}: слияние {best}, размер словаря: {len(vocab)}')
print(f'Обучение завершено. Размер словаря: {len(vocab)}')

Итерация 0: слияние ('т', 'о'), размер словаря: 35
Итерация 10: слияние ('л', 'и'), размер словаря: 45
Итерация 20: слияние ('л', 'ь'), размер словаря: 55
Итерация 30: слияние ('в', 'е'), размер словаря: 65
Итерация 40: слияние ('п', 'ро'), размер словаря: 75
Итерация 50: слияние ('р', 'у'), размер словаря: 85
Итерация 60: слияние ('р', 'ы'), размер словаря: 95
Итерация 70: слияние ('ни', 'я'), размер словаря: 105
Итерация 80: слияние ('ст', 'и'), размер словаря: 115
Итерация 90: слияние ('л', 'у'), размер словаря: 125
Итерация 100: слияние ('и', 'с'), размер словаря: 135
Итерация 110: слияние ('б', 'у'), размер словаря: 145
Итерация 120: слияние ('с', 'у'), размер словаря: 155
Итерация 130: слияние ('бо', 'ль'), размер словаря: 165
Итерация 140: слияние ('д', 'ы'), размер словаря: 175
Итерация 150: слияние ('ст', 'ра'), размер словаря: 185
Итерация 160: слияние ('пе', 'р'), размер словаря: 195
Итерация 170: слияние ('е', 'го'), размер словаря: 205
Итерация 180: слияние ('хо', 'ди'), р

In [13]:
vocab

defaultdict(int,
            {'я': 342793,
             '_ed': 3064170,
             'д': 505746,
             'у': 397444,
             'м': 640711,
             'а': 1251411,
             'л': 675358,
             'б': 268953,
             'ч': 272410,
             'и': 1249335,
             'т': 1200315,
             'ь': 318961,
             'е': 1380256,
             'с': 861114,
             'в': 702425,
             'о': 1867683,
             'х': 161763,
             'к': 515989,
             'р': 728335,
             'ы': 371311,
             'з': 279298,
             'н': 1071319,
             'й': 180365,
             'ю': 131659,
             'ж': 163408,
             'г': 251697,
             'п': 435103,
             'щ': 63687,
             'ш': 108749,
             'ё': 48462,
             'ц': 66405,
             'э': 112219,
             'ф': 44082,
             'ъ': 4417,
             'то': 313608,
             'но': 204957,
             'на': 204786,
             'с

In [14]:
print(len(vocab))

1000


In [15]:
def build_token_mappings(vocab):
    tokens = list(vocab.keys())
    token_to_id = {token: idx for idx, token in enumerate(tokens)}
    id_to_token = {idx: token for idx, token in enumerate(tokens)}
    return token_to_id, id_to_token

token_to_id, id_to_token = build_token_mappings(vocab)

In [16]:
def add_special_tokens(token_to_id, id_to_token, special_tokens):
    for token in special_tokens:
        if token not in token_to_id:
            idx = len(token_to_id)
            token_to_id[token] = idx
            id_to_token[idx] = token

special_tokens = ['<start>', '<end>', '<unk>', '<pad>']
add_special_tokens(token_to_id, id_to_token, special_tokens)

print(f"\nДобавлены специальные токены. Размер словаря token_to_id: {len(token_to_id)}")


Добавлены специальные токены. Размер словаря token_to_id: 1004


In [17]:
import json
import os

output_dir = "C:/Tokenizer/Vocab/RuVocab" 

os.makedirs(output_dir, exist_ok=True)

with open(os.path.join(output_dir, 'token_to_id.json'), 'w', encoding='utf-8') as out_file:
    json.dump(token_to_id, out_file, indent=6, ensure_ascii=False)

with open(os.path.join(output_dir, 'id_to_token.json'), 'w', encoding='utf-8') as out_file:
    json.dump(id_to_token, out_file, indent=6, ensure_ascii=False)

print(f"Словари token_to_id и id_to_token сохранены в {output_dir}")


Словари token_to_id и id_to_token сохранены в C:/Tokenizer/Vocab/RuVocab


In [18]:
def tokenize_word(word, vocab):
    symbols = list(word) + ['_ed']  
    i = 0
    tokens = []
    while i < len(symbols):
        j = len(symbols)
        found = False
        while j > i:
            substring = ''.join(symbols[i:j])
            if substring in vocab:
                tokens.append(substring)
                i = j
                found = True
                break
            else:
                j -= 1
        if not found:
            tokens.append('<unk>')
            i += 1
    return tokens

In [19]:
def encode_text(text, token_to_id, vocab):
    token_ids = []
    for word in text.split():
        word_tokens = tokenize_word(word, vocab)
        for token in word_tokens:
            token_id = token_to_id.get(token)
            if token_id is not None:
                token_ids.append(token_id)
            else:
                token_ids.append(token_to_id.get('<unk>'))
    return token_ids

In [20]:
def decode_ids(token_ids, id_to_token):
    tokens = [id_to_token.get(token_id, '<unk>') for token_id in token_ids]
    words = []
    word = ''
    for token in tokens:
        if token == '_ed':
            words.append(word)
            word = ''
        elif token not in ['<start>', '<end>', '<pad>']:
            word += token
    if word:
        words.append(word)
    return ' '.join(words)

In [21]:
test_text = "Я учусь в техническом вузе, чтобы стать программистом"
test_text = clean_text(test_text)

In [22]:
encoded_ids = encode_text(test_text, token_to_id, vocab)
print("\nКодированные идентификаторы:")
print(encoded_ids)


Кодированные идентификаторы:
[0, 1, 3, 238, 105, 1, 14, 1, 324, 38, 378, 4, 1, 165, 566, 1, 190, 1, 95, 49, 1, 821, 68, 98, 4, 1]


In [23]:
decoded_text = decode_ids(encoded_ids, id_to_token)
print("\nДекодированный текст:")
print(decoded_text)


Декодированный текст:
я учусь в техническом вузе чтобы стать программистом
